In [1]:
from keras.layers import Input, Dense, Conv2D, Flatten, MaxPooling2D, Reshape, UpSampling2D, BatchNormalization, AveragePooling2D, Activation, Lambda, Conv2DTranspose, ReLU
from keras.models import Model, Sequential
from keras.regularizers import l1

in_height = 720//2
in_width =  1280//2

#out_height = 720
#out_width = 1280

out_height = in_height
out_width = in_width

Using TensorFlow backend.


In [2]:
from keras.constraints import maxnorm
from keras.losses import *
encoder_input =  Input(shape=(in_height,in_width,3))

max_norm = 3
# Encoder Layers
encoder = Conv2D(8, (3, 3),padding='same', kernel_constraint=maxnorm(max_norm))(encoder_input)
encoder = Activation('relu')(encoder)
encoder = AveragePooling2D((2, 2), padding='same',)(encoder)
encoder = Conv2D(16, (3, 3), padding='same', kernel_constraint=maxnorm(max_norm))(encoder)
encoder = Activation('relu')(encoder)
encoder = AveragePooling2D((2, 2), padding='same')(encoder)

encoder = Conv2D(32, (3, 3), padding='same', kernel_constraint=maxnorm(max_norm))(encoder)
encoder = Activation('relu')(encoder)
encoder = AveragePooling2D((2, 2), padding='same')(encoder)

encoder =  Flatten(name='flatten_1')(encoder)

encoder_model = Model(encoder_input,encoder)
encoder_model.summary()

#encoder_model.load_weights("16C-360x640-encoder.h5")

# Decoder Layers
decoder_input = Input(shape=(45 * 80 * 32,))
decoder = Reshape(( 45, 80, 32))(decoder_input)
decoder = Conv2DTranspose(16, (3, 3), padding='same',strides=2, kernel_constraint=maxnorm(max_norm))(decoder)
decoder = Activation('relu')(decoder)
decoder = Conv2DTranspose(16, (3, 3), padding='same',strides=2, kernel_constraint=maxnorm(max_norm))(decoder)
decoder = Activation('relu')(decoder)
decoder = Conv2DTranspose(8, (3, 3), padding='same',strides=2, kernel_constraint=maxnorm(max_norm))(decoder)
decoder = Conv2D(3, (1, 1), padding='same')(decoder)

#decoder = ReLU(max_value=255)(decoder)
decoder_model = Model(decoder_input,decoder)
decoder_model.summary()

autoencoder = Model(encoder_input,decoder_model(encoder))
autoencoder.compile(optimizer='adam',  loss=logcosh,metrics=['acc'])

autoencoder.load_weights('weights-improvement-01-0.87.hdf5')

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 360, 640, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 360, 640, 8)       224       
_________________________________________________________________
activation_1 (Activation)    (None, 360, 640, 8)       0         
_________________________________________________________________
average_pooling2d_1 (Average (None, 180, 320, 8)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 180, 320, 16)      1168      
_________________________________________________________________
activation_2 (Activation)    (None, 180, 320, 16)      0         
_________________________________________________________________
average_pooling2d_2 (Average (None, 90, 160, 16)       0   

In [9]:
from keras.models import load_model

encoder_model = load_model("16C-360x640-encoder.h5")
decoder_model = load_model("16C-360x640-decoder.h5")

autoencoder = Model(encoder_model.inputs,decoder_model(encoder_model.outputs))
autoencoder.summary()

autoencoder.compile(optimizer='adam',  loss=logcosh,metrics=['acc'])

C:\Users\mayan\Anaconda3\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 360, 640, 3)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 360, 640, 8)       224       
_________________________________________________________________
activation_22 (Activation)   (None, 360, 640, 8)       0         
_________________________________________________________________
average_pooling2d_16 (Averag (None, 180, 320, 8)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 180, 320, 16)      1168      
_________________________________________________________________
activation_23 (Activation)   (None, 180, 320, 16)      0         
_________________________________________________________________
average_pooling2d_17 (Averag (None, 90, 160, 16)       0   

In [ ]:
tfjs.converters.save_keras_model(autoencoder, './tensorflowjs-model/')

In [3]:
import cv2, os
import numpy as np
#import pandas as pd

train_dir = "../Datasets/DIV2K_train_HR/"
valid_dir = "../Datasets/DIV2K_valid_HR/"

X = list()
Y = list()
#files = pd.read_csv('image_data.csv').Path
train_files = os.listdir(train_dir)
valid_files = os.listdir(valid_dir)

def getImage(file):
    #‪C:\Users\mayan\Downloads\2-dog.jpg
    frame = cv2.imread(file,1)
    return [np.array([cv2.resize(frame,(in_width,in_height))]),np.array([cv2.resize(frame,(out_width,out_height))])]

def getValidation():
    x_valid,y_valid = [],[]   
    for file in valid_files:
        frame = cv2.imread(valid_dir + file,1)
        try:
            #y_valid.append(cv2.resize(frame,(out_width,out_height)))
            x_valid.append(cv2.resize(frame,(in_width,in_height)))
        except Exception as ex:
            print(ex)
    return np.array(x_valid),np.array(y_valid)

"""
for file in train_files:
    frame = cv2.imread(train_dir + file,1)
    try:
        X.append(cv2.resize(frame,(in_width,in_height)))
    except Exception as ex:
        print(ex)
"""
x_valid,y_valid = getValidation()

In [ ]:
encoded_output = encoder_model.predict(X)

In [ ]:
np.save("images.npy",X)
np.save("encoded_output.npy",encoded_output)

In [7]:
train_files = os.listdir("./../Datasets/flickr30k_images/")
for file in train_files:
    frame = cv2.imread( './../Datasets/flickr30k_images/'+ file,1)
    try:
        height, width, channels = frame.shape
        if height >= in_height and width >= in_width:
            X.append(cv2.resize(frame,(in_width,in_height)))
    except Exception as ex:
        print(ex)
    
X = np.array(X)

'NoneType' object has no attribute 'shape'
'NoneType' object has no attribute 'shape'


In [4]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True)
train_generator = datagen.flow_from_directory(
        'D:\\Fast and Furious',
        target_size=(in_height, in_width),
        batch_size=1,
        shuffle=True,
        class_mode="input")

filepath="weights-improvement-{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

Found 7078 images belonging to 3 classes.


In [ ]:
from IPython.display import clear_output
from keras.optimizers import Adam, Adadelta

validation_accuracy = 0.7050989866256714
learning_rate = (1-validation_accuracy)/10000

#it = datagen.flow(np.array(X),batch_size=512,shuffle = True)

x_test = getImage('C:/Users/mayan/Desktop/tensorflow-autoencoder/static/pexels-photo-257840.jpeg')[0];

while True:
    """
    X_ = next(train_generator)[0]  
    hist = autoencoder.fit(X_,
                epochs=10,
                batch_size=8,validation_data = (x_valid,x_valid))
    """
    autoencoder.fit_generator(train_generator,steps_per_epoch=7078 * 4,validation_data=(x_valid,x_valid),epochs=1,callbacks=[checkpoint])
    clear_output()
    decoded_imgs = autoencoder.predict(x_test)
    h = autoencoder.evaluate(x_test,x_test)
    
    print("Test Loss",h[0])
    print("Test Accuracy",h[1])
    
    cv2.imwrite("./restructured/test_image.jpg" ,decoded_imgs[0])
    cv2.imwrite("./restructured/original-test_image.jpg" ,x_test[0])
    
    decoder_model.save("32C-%dx%d-decoder.h5"%(out_height,out_width))
    encoder_model.save("32C-%dx%d-encoder.h5"%(in_height,in_width))

1/1 [==============================] - 0s 12ms/step
Test Loss 2.6306862831115723
Test Accuracy 0.9407768845558167
Epoch 1/1
24757/28312 [=========================>....] - ETA: 2:14 - loss: 3.9653 - acc: 0.8881

In [ ]:
it = datagen.flow(x_test,batch_size=1)

In [33]:
decoded_imgs = next(train_generator)
cv2.imshow("Desired", decoded_imgs[1][0]/255) 
cv2.imshow("Reconstructed", decoded_imgs[0][0]/255) 


cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
x_test = getImage('C:/Users/mayan/Desktop/tensorflow-autoencoder/static/pexels-photo-257840.jpeg')[0];
#x_test = getImage(train_dir + 'IMG_20191018_232049.jpg')[0]
#x_test = x_valid[4:5]

#decoded_imgs = autoencoder.predict(x_test)
import sys

decoded_imgs = encoder_model.predict(x_test)
decoded_imgs = decoded_imgs/np.max(decoded_imgs) * 255
decoded_imgs  = decoded_imgs.astype('uint8')

decoded_imgs = decoder_model.predict(decoded_imgs)
decoded_imgs = decoded_imgs/np.max(decoded_imgs)


cv2.imshow("Reconstructed", decoded_imgs[0]) 
cv2.imshow("Original", x_test[0]) 

cv2.waitKey(0)
cv2.destroyAllWindows()

In [27]:
encoder_model.save("encoder-final-model.h5")
decoder_model.save("decoder-final-model.h5")

In [ ]:
train_files = os.listdir("C:/Users/mayan/Machine Learning/Datasets/Events/")
for file in train_files:
    frame = cv2.imread("C:/Users/mayan/Machine Learning/Datasets/Events/" + file,1)
    try:
        height, width, channels = frame.shape
        if height >= in_height and width >= in_width:
            X.append(cv2.resize(frame,(in_width,in_height)))
    except Exception as ex:
        print(ex)

In [ ]:
h = autoencoder.evaluate(x_test,x_test)
#autoencoder.metrics_names
h

In [ ]:
"""
{'learning_rate': 2.9490101951523684e-05,
 'beta_1': 0.8999999761581421,
 'beta_2': 0.9990000128746033,
 'decay': 0.0,
 'epsilon': 1e-07,
 'amsgrad': False}
"""

autoencoder.optimizer.get_config()

In [ ]:
##Save Model
json= decoder_model.to_json()
with open("decoder.json", "w") as json_file:
    json_file.write(json)

decoder_model.save_weights("decoder.h5")

json= encoder_model.to_json()
with open("encoder.json", "w") as json_file:
    json_file.write(json)
encoder_model.save_weights("encoder.h5")

print("Saved model to disk")

In [ ]:
#x_test,y_test = next(getData(16))
x_valid,y_valid = getImage('C:/Users/mayan/Downloads/2-dog.jpg')
#x_test,y_test = getImage('dark-fantasy-wallpapers-28123-6689698.jpg')


In [ ]:
import matplotlib.pyplot as plt
import imutils, time

%matplotlib inline

num_images = 10
np.random.seed(42)

random_test_images = np.random.randint(x_valid.shape[0], size=min(num_images,x_valid.size))

encoded_imgs = encoder_model.predict(x_valid)
decoded_imgs = decoder_model.predict(encoded_imgs)

#decoded_imgs = autoencoder.predict(x_test)

plt.figure(figsize=(60, 50))

for i, image_idx in enumerate(random_test_images):
    # plot original image
    ax = plt.subplot(3, num_images, i + 1)
    plt.imshow(x_valid[image_idx])
    #plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    cv2.imwrite("./real/%s.jpg" %time.time(),x_valid[image_idx])
    '''
    # plot encoded image
    ax = plt.subplot(3, num_images, num_images + i + 1)
    plt.imshow(encoded_imgs[image_idx].reshape(32, 16))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    '''
    # plot reconstructed image
    ax = plt.subplot(3, num_images, num_images + i + 1)
    name = time.time()
    cv2.imwrite("./restructured/%s.jpg" %name,decoded_imgs[image_idx] )
    cv2.imwrite("./restructured/original-%s.jpg" %name,x_valid[image_idx] )
    plt.imshow(decoded_imgs[image_idx]/255)
    #plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
plt.show()